In [2]:
!curl -fsSL https://ollama.com/install.sh | sh

>>> Downloading ollama...
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
!ollama

Usage:
  ollama [flags]
  ollama [command]

Available Commands:
  serve       Start ollama
  create      Create a model from a Modelfile
  show        Show information for a model
  run         Run a model
  pull        Pull a model from a registry
  push        Push a model to a registry
  list        List models
  ps          List running models
  cp          Copy a model
  rm          Remove a model
  help        Help about any command

Flags:
  -h, --help      help for ollama
  -v, --version   Show version information

Use "ollama [command] --help" for more information about a command.


In [4]:
import os
import asyncio

# NB: You may need to set these depending and get cuda working depending which backend you are running.
# Set environment variable for NVIDIA library
# Set environment variables for CUDA
os.environ['PATH'] += ':/usr/local/cuda/bin'
# Set LD_LIBRARY_PATH to include both /usr/lib64-nvidia and CUDA lib directories
os.environ['LD_LIBRARY_PATH'] = '/usr/lib64-nvidia:/usr/local/cuda/lib64'

async def run_process(cmd):
    print('>>> starting', *cmd)
    process = await asyncio.create_subprocess_exec(
        *cmd,
        stdout=asyncio.subprocess.PIPE,
        stderr=asyncio.subprocess.PIPE
    )

    # define an async pipe function
    async def pipe(lines):
        async for line in lines:
            print(line.decode().strip())

        await asyncio.gather(
            pipe(process.stdout),
            pipe(process.stderr),
        )

    # call it
    await asyncio.gather(pipe(process.stdout), pipe(process.stderr))

In [9]:
# prompt: NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.

!sudo apt-get update
!sudo apt-get install nvidia-driver-470
!sudo reboot

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
nvidia-driver-470 is already the newest version (470.239.06-0ubuntu0.22.04.1).
The following packages were automatically installed and are

In [5]:
!nvidia-smi

Mon Jun 17 16:21:02 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   57C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
import asyncio
import threading

async def start_ollama_serve():
    await run_process(['ollama', 'serve'])

def run_async_in_thread(loop, coro):
    asyncio.set_event_loop(loop)
    loop.run_until_complete(coro)
    loop.close()

# Create a new event loop that will run in a new thread
new_loop = asyncio.new_event_loop()

# Start ollama serve in a separate thread so the cell won't block execution
thread = threading.Thread(target=run_async_in_thread, args=(new_loop, start_ollama_serve()))
thread.start()

>>> starting ollama serve
Couldn't find '/root/.ollama/id_ed25519'. Generating new private key.
Your new public key is:

ssh-ed25519 AAAAC3NzaC1lZDI1NTE5AAAAIN+tSIM+jZlLAVTvVG9pt/8xE5UvLWkSsxrqWK5ph8rL

2024/06/17 16:21:08 routes.go:1011: INFO server config env="map[OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_HOST:http://127.0.0.1:11434 OLLAMA_KEEP_ALIVE: OLLAMA_LLM_LIBRARY: OLLAMA_MAX_LOADED_MODELS:1 OLLAMA_MAX_QUEUE:512 OLLAMA_MAX_VRAM:0 OLLAMA_MODELS:/root/.ollama/models OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:1 OLLAMA_ORIGINS:[http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://*] OLLAMA_RUNNERS_DIR: OLLAMA_TMPDIR:]"
time=2024-06-17T16:21:08.318Z level=INFO source=images.go:725 msg="total blobs: 0"
time=2024-06-17T16:21:08.319Z level=INFO source=images.go:732 msg=

In [11]:
!ollama pull phi3

[GIN] 2024/06/17 - 16:22:07 | 200 |      23.811µs |       127.0.0.1 | HEAD     "/"
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ time=2024-06-17T16:22:10.285Z level=INFO source=download.go:136 msg="downloading b26e6713dc74 in 24 100 MB part(s)"
pulling manifest ⠹ pulling manifest 
pulling b26e6713dc74...   0% ▕▏    0 B/2.4 GB                  pulling manifest 
pulling b26e6713dc74...   0% ▕▏    0 B/2.4 GB                  pulling manifest 
pulling b26e6713dc74...   0% ▕▏    0 B/2.4 GB                  pulling manifest 
pulling b26e6713dc74...   0% ▕▏    0 B/2.4 GB                  pulling manifest 
pulling b26e6713dc74..

In [7]:
!pip install pyautogen -qqq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.2/290.2 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 5.6 MB/s eta 0:00:00
time=2024-06-17T16:21:18.946Z level=INFO source=payload.go:44 msg="Dynamic LLM libraries [cpu cpu_avx cpu_avx2 cuda_v11 rocm_v60002]"
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 5.0 MB/s eta 0:00:00
time=2024-06-17T16:21:19.392Z level=INFO source=types.go:71 msg="inference compute" id=GPU-c26b57e6-1c74-99b0-15c0-9f073f7ea38c library=cuda compute=7.5 driver=12.2 name="Tesla T4" total="14.7 GiB" available="14.6 GiB"
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 325.5/325.5 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.4 MB/s eta 0:00:0

In [5]:
# os.environ.update({'OLLAMA_HOST': '127.0.0.1'})

In [6]:
!ollama run phi3

[GIN] 2024/06/17 - 15:36:33 | 200 |      68.019µs |       127.0.0.1 | HEAD     "/"
[GIN] 2024/06/17 - 15:36:33 | 200 |    2.945498ms |       127.0.0.1 | POST     "/api/show"
[GIN] 2024/06/17 - 15:36:33 | 200 |     605.282µs |       127.0.0.1 | POST     "/api/show"
⠙ time=2024-06-17T15:36:33.936Z level=INFO source=memory.go:133 msg="offload to gpu" layers.requested=-1 layers.real=33 memory.available="9.6 GiB" memory.required.full="3.1 GiB" memory.required.partial="3.1 GiB" memory.required.kv="768.0 MiB" memory.weights.total="2.2 GiB" memory.weights.repeating="2.1 GiB" memory.weights.nonrepeating="77.1 MiB" memory.graph.full="128.0 MiB" memory.graph.partial="128.0 MiB"
time=2024-06-17T15:36:33.936Z level=INFO source=server.go:341 msg="starting llama server" cmd="/tmp/ollama2172352399/runners/cpu_avx2/ollama_llama_server --model /root/.ollama/models/blobs/sha256-b26e6713dc749dda35872713fa19a568040f475cc71cb132cff332fe7e216462 --ctx-size 2048 --batch-size 512 --embedding --log-disable --pa

In [9]:
!pip install ollama

In [12]:
# !nohup ollama serve &
# !ollama run phi3
import ollama

stream = ollama.chat(
    model="phi3",
    messages=[{"role": "user", "content": "Why is the sky blue?"}],
    stream=True,
)

for chunk in stream:
    print(chunk["message"]["content"], end="", flush=True)


time=2024-06-17T16:22:48.615Z level=INFO source=memory.go:133 msg="offload to gpu" layers.requested=-1 layers.real=33 memory.available="14.6 GiB" memory.required.full="3.6 GiB" memory.required.partial="3.6 GiB" memory.required.kv="768.0 MiB" memory.weights.total="2.2 GiB" memory.weights.repeating="2.1 GiB" memory.weights.nonrepeating="77.1 MiB" memory.graph.full="128.0 MiB" memory.graph.partial="128.0 MiB"
time=2024-06-17T16:22:48.616Z level=INFO source=memory.go:133 msg="offload to gpu" layers.requested=-1 layers.real=33 memory.available="14.6 GiB" memory.required.full="3.6 GiB" memory.required.partial="3.6 GiB" memory.required.kv="768.0 MiB" memory.weights.total="2.2 GiB" memory.weights.repeating="2.1 GiB" memory.weights.nonrepeating="77.1 MiB" memory.graph.full="128.0 MiB" memory.graph.partial="128.0 MiB"
time=2024-06-17T16:22:48.616Z level=INFO source=server.go:341 msg="starting llama server" cmd="/tmp/ollama1114863355/runners/cuda_v11/ollama_llama_server --model /root/.ollama/mode

In [25]:
# !nohup ollama serve &
from autogen import AssistantAgent, UserProxyAgent

config_list = [
  {
    "model": "phi3",
    "base_url": "http://localhost:11434/v1",
    "api_key": "ollama",
  }
]

assistant = AssistantAgent("assistant", llm_config={"config_list": config_list})

user_proxy = UserProxyAgent("user_proxy", code_execution_config={"work_dir": "coding", "use_docker": False})
user_proxy.initiate_chat(assistant, message="Plot a chart of NVDA and TESLA stock price change YTD.")

user_proxy (to assistant):

Plot a chart of NVDA and TESLA stock price change YTD.

--------------------------------------------------------------------------------
assistant (to user_proxy):

 #!/usr/bin/env python3
import matplotlib.pyplot as plt
import yfinance as yf  # Install with pip install yfinance if not already installed

# Get historical data for NVDA and Tesla (stock ticker: TSLA)
nvidia_ticker = 'NVDA'
tesla_ticker = 'TSLA'
start_date = '2021-01-01'  # Change this according to YTD start date
end_date = None  # Automatically uses today's date for the current year
nvidia_stock = yf.Ticker(nvidia_ticker)
tesla_stock = yf.Ticker(tesla_ticker)
nvidia_data = nvidia_stock.history(start=start_date, end=end_date)
tesla_data = tesla_stock.history(start=start_date, end=end_date)

# Calculate the year-to-date (YTD) stock price change for both NVDA and TSLA
nvidia_ytd = nvidia_data['Close'].pct_change().iloc[-1] * 100  # Convert to percentage
tesla_ytd = tesla_data['Close'].pct_change(

APIConnectionError: Connection error.

In [26]:
!ls -a

.  ..  .cache  .config	sample_data


In [27]:
# !nohup ollama serve &
!curl http://localhost:11434/api/chat -d '{ "model": "phi3", "stream": true, "messages": [{ "role": "user", "content": "Why is the sky green?" }]}'

curl: (7) Failed to connect to localhost port 11434 after 0 ms: Connection refused
